In [10]:
!pip install tushare

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tushare as ts
from sklearn import preprocessing

In [12]:
ts.set_token('b11ee55fd1f2c78f551b05ad1ec41b336a8d8b8c75f3a2f2ce5d0ea4')   #需要在 tushare 官网申请一个账号，然后得到 token 后才能通过数据接口获取数据
pro = ts.pro_api()

df = pro.daily(ts_code='000001.SZ', start_date='2015-01-01', end_date='2020-02-25')

df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20191231,16.57,16.63,16.31,16.45,16.57,-0.12,-0.7242,704442.25,1154704.348
1,000001.SZ,20191230,16.46,16.63,16.10,16.57,16.63,-0.06,-0.3608,976970.31,1603152.786
2,000001.SZ,20191227,16.53,16.93,16.43,16.63,16.47,0.16,0.9715,1042574.72,1741473.179
3,000001.SZ,20191226,16.34,16.48,16.32,16.47,16.30,0.17,1.0429,372033.86,610381.757
4,000001.SZ,20191225,16.45,16.56,16.24,16.30,16.40,-0.10,-0.6098,414917.98,679664.596


In [13]:
df = df.iloc[::-1]
df.reset_index(inplace=True)

training_set = df.loc[:, ['close']]

training_set = training_set.values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

X_train = []
y_train = []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i])
    y_train.append(training_set_scaled[i, training_set_scaled.shape[1] - 1])
X_train, y_train = np.array(X_train), np.array(y_train)

In [14]:

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

regressor = Sequential()
 
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))
 
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
 
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/10
37/37 [==============================] - 12s 123ms/step - loss: 0.0323
Epoch 2/10
37/37 [==============================] - 5s 125ms/step - loss: 0.0073
Epoch 3/10
37/37 [==============================] - 5s 123ms/step - loss: 0.0053
Epoch 4/10
37/37 [==============================] - 5s 125ms/step - loss: 0.0044
Epoch 5/10
37/37 [==============================] - 5s 129ms/step - loss: 0.0041
Epoch 6/10
37/37 [==============================] - 5s 128ms/step - loss: 0.0040
Epoch 7/10
37/37 [==============================] - 5s 126ms/step - loss: 0.0039
Epoch 8/10
37/37 [==============================] - 5s 125ms/step - loss: 0.0036
Epoch 9/10
37/37 [==============================] - 5s 124ms/step - loss: 0.0039
Epoch 10/10
37/37 [==============================] - 5s 124ms/step - loss: 0.0037


In [41]:
import tushare as ts

ts.set_token('b11ee55fd1f2c78f551b05ad1ec41b336a8d8b8c75f3a2f2ce5d0ea4')
pro = ts.pro_api()

df_test = pro.daily(ts_code='000001.SZ', start_date='2018-01-01', end_date='2021-01-01')
df_test = pd.DataFrame(df_test, columns = ['ts_code','trade_date','close','open','high','low'])


df_test = df_test.iloc[::-1]
df_test.reset_index(inplace=True)


dataset_test = df_test.loc[:, ['close']]
dataset_total = pd.concat((df_test[['close']],df[['close']]))

inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1, dataset_test.shape[1])

inputs = sc.transform(inputs)

predicted_stock_price = []

#准备测试数据，就是把要测试的数据和以前训练的数据结合起来组装出要测试的 X，因为是要利用过去 60 个交易日的数据，只靠一个交易日的收盘价是不够的
X_test = []

for i in range(60, 60 + len(dataset_test)):
    X_test.append(inputs[i-60:i])
X_test = np.array(X_test)

# print(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], dataset_test.shape[1]))

predicted_stock_price = regressor.predict(X_test)

#再把规则化数据转回成正常的价格数据，现在就可以得出预测的下个交易日收盘价格
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

print(predicted_stock_price)

[[ 9.383341 ]
 [ 9.36744  ]
 [ 9.359534 ]
 [ 9.358493 ]
 [ 9.362842 ]
 [ 9.370969 ]
 [ 9.381214 ]
 [ 9.392215 ]
 [ 9.4030485]
 [ 9.412874 ]
 [ 9.421195 ]
 [ 9.4280205]
 [ 9.433673 ]
 [ 9.438971 ]
 [ 9.444634 ]
 [ 9.451623 ]
 [ 9.460418 ]
 [ 9.471768 ]
 [ 9.485199 ]
 [ 9.500212 ]
 [ 9.516053 ]
 [ 9.531867 ]
 [ 9.546948 ]
 [ 9.560967 ]
 [ 9.574666 ]
 [ 9.58871  ]
 [ 9.603946 ]
 [ 9.620907 ]
 [ 9.638744 ]
 [ 9.658081 ]
 [ 9.679681 ]
 [ 9.703736 ]
 [ 9.729046 ]
 [ 9.7537775]
 [ 9.775365 ]
 [ 9.7923   ]
 [ 9.804155 ]
 [ 9.810539 ]
 [ 9.8111725]
 [ 9.806886 ]
 [ 9.799063 ]
 [ 9.788873 ]
 [ 9.776895 ]
 [ 9.763831 ]
 [ 9.75091  ]
 [ 9.739319 ]
 [ 9.730391 ]
 [ 9.725495 ]
 [ 9.722948 ]
 [ 9.719659 ]
 [ 9.7129135]
 [ 9.70033  ]
 [ 9.68135  ]
 [ 9.656766 ]
 [ 9.627718 ]
 [ 9.595542 ]
 [ 9.561714 ]
 [ 9.527479 ]
 [ 9.4950485]
 [ 9.467013 ]
 [ 9.445232 ]
 [ 9.430562 ]
 [ 9.4226885]
 [ 9.420232 ]
 [ 9.421133 ]
 [ 9.42343  ]
 [ 9.425181 ]
 [ 9.425252 ]
 [ 9.42267  ]
 [ 9.415441 ]
 [ 9.402295 ]
 [ 9.3